In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings

warnings.filterwarnings(action='ignore')

In [3]:
def data_process(filename1, filename2):
    vix_df = pd.read_excel(filename1).dropna()
    vix_df = vix_df.rename(columns={'美国:标准普尔500波动率指数(VIX)': 'VIX', '美国:标准普尔500指数': 'SPX500',
                                    '指标名称': 'Date'}).set_index('Date')

    contract_df = pd.read_excel(filename2, index_col=0)

    cols = contract_df.columns
    select_col = [x for x in cols if x[-3:] != 'POS']
    select_col = select_col[:-6]
    df = contract_df[select_col]

    df = df.dropna(how='all')

    df = pd.merge(df, vix_df, left_index=True, right_index=True, how='inner')
    df = df['2007-01-01':]

    months = list(np.unique([m[:-3] for m in df.index.astype(str).tolist()]))

    df_process = pd.DataFrame()
    for m in months[:-5]:
        sub_df = df[m]
        if m < '2020-01':
            if int(m[2:4]) < 9:
                col = [x for x in select_col if m[2:4] in x] + [x for x in select_col if
                                                                ('0' + str(int(m[2:4]) + 1)) in x][:7]
            if int(m[2:4]) < 19:
                col = [x for x in select_col if m[2:4] in x] + [x for x in select_col if str(int(m[2:4]) + 1) in x][:7]
            if int(m[2:4]) == 19:
                col = [x for x in select_col if m[2:4] in x] + select_col[-12:-5]
        else:
            col = select_col[-12:]

        sub = sub_df[col]
        m1 = int(m[-2:])
        m2 = int(m[-2:]) + 1
        m3 = int(m[-2:]) + 2
        m4 = int(m[-2:]) + 3
        m7 = int(m[-2:]) + 6
        sub_df['AVG'] = sub.iloc[:, [m1, m2, m3]].mean(axis=1)
        sub_df['COMPAR'] = (sub.iloc[:, m3] - sub.iloc[:, m1]) - (sub.iloc[:, m7] - sub.iloc[:, m4])
        df_process = df_process.append(sub_df)

    df_process['TRI_S'] = ((df_process['VIX'] < df_process['AVG']).astype(int)).shift(1).rolling(3).sum().fillna(0)
    df_process['LQD_S'] = ((df_process['VIX'] >= df_process['AVG']).astype(int)).shift(1).rolling(3).sum().fillna(0)
    df_process['Signal'] = np.where(df_process['COMPAR'] > 0.2, 0, 0.5)
    df_process['Signal'].loc[df_process['COMPAR'] < -0.2] = 1
    df_process['Ratio_L'] = (df_process['Signal'].rolling(5).mean() * 0.25 + 0.75).fillna(0.75)

    df_process['LONG'] = InitialPos * df_process['Ratio_L']
    df_process['SHORT'] = 0
    df_process['Total_pos'] = [[0.]*7] * len(df_process)
    df_process['Weight_pos'] = [[0.]*7] * len(df_process)
    return df_process


def heldging_func(df, initial_short_pos=0):
    num_days = len(df)
    dates = df.index.tolist()
    short_roll = 1 / num_days
    long_roll = 1 / num_days
    total_pos, weight_pos = [], []
    for i in range(num_days):
        if i == 0:
            df.loc[dates[i], 'SHORT'] = initial_short_pos
            if df.loc[dates[i], 'TRI_S'] == 3:
                df.loc[dates[i], 'SHORT'] = min(InitialPos * 0.75, df.loc[dates[i], 'SHORT'] + InitialPos * 0.25)
        if i != 0:
            # 最后5个交易日不交易
            if df.loc[dates[i], 'TRI_S'] == 3 and (num_days - i) > 5:
                df.loc[dates[i], 'SHORT'] = min(InitialPos * 0.75, df.iloc[i - 1]['SHORT'] + InitialPos * 0.25)
            elif df.iloc[i]['LQD_S'] == 3 and (num_days - i) > 5:
                df.loc[dates[i], 'SHORT'] = max(0, df.iloc[i - 1]['SHORT'] - InitialPos * 0.25)
            else:
                df.loc[dates[i], 'SHORT'] = df.loc[dates[i - 1], 'SHORT']

        # 空头展仓
        short1, short2, short3 = 0, 0, 0
        if df.loc[dates[i], 'SHORT'] != 0:
            # print(df.loc[dates[i], 'SHORT'])
            short3 = df.loc[dates[i], 'SHORT'] / 2 * (i + 1) * short_roll
            short1 = df.loc[dates[i], 'SHORT'] / 2 - short3
            short2 = df.loc[dates[i], 'SHORT'] / 2

            # 以下是list弱赋值，无法实现
            # df.loc[dates[i], 'Total_pos'][2] = df.loc[dates[i], 'SHORT'] / 2 * (i + 1) * short_roll
            # df.loc[dates[i], 'Total_pos'][0] = df.loc[dates[i], 'SHORT'] / 2 - df.loc[dates[i], 'Total_pos'][2]
            # df.loc[dates[i], 'Total_pos'][1] = df.loc[dates[i], 'SHORT'] / 2

        # 多头展仓
        long5 = df.loc[dates[i], 'LONG'] / 3
        long6 = df.loc[dates[i], 'LONG'] / 3
        long7 = df.loc[dates[i], 'LONG'] / 3 * (i + 1) * long_roll
        long4 = df.loc[dates[i], 'LONG'] / 3 - long7

        # 以下是list弱赋值，无法实现
        # df.loc[dates[i], 'Total_pos'][4] = df.loc[dates[i], 'LONG'] / 3
        # df.loc[dates[i], 'Total_pos'][5] = df.loc[dates[i], 'LONG'] / 3
        # df.loc[dates[i], 'Total_pos'][6] = df.loc[dates[i], 'LONG'] / 3 * (i + 1) * long_roll
        # df.loc[dates[i], 'Total_pos'][3] = df.loc[dates[i], 'LONG'] / 3 - df.loc[dates[i], 'Total_pos'][6]

        total_pos.append([short1, short2, short3, long4, long5, long6, long7])
        weight_pos.append([x/(df.loc[dates[i], 'LONG'] + df.loc[dates[i], 'SHORT']) for x in [-short1, -short2, -short3, long4, long5, long6, long7]])
    initial_short_pos = df.iloc[-1]['SHORT']
    return df, initial_short_pos, total_pos, weight_pos

In [8]:
InitialPos = 100
filename1 = 'DATA/市场波动率指数(VIX).xls'
filename2 = 'DATA/CT_Data.xlsx'

df = data_process(filename1, filename2)
months = list(np.unique([m[:-3] for m in df.index.astype(str).tolist()]))
initial_short_pos = 0

out_df = pd.DataFrame()
for m in tqdm(months):
    sub_df = df[m]
    sub_df, initial_short_pos, total_pos, weight_pos = heldging_func(sub_df, initial_short_pos)
    sub_df['Total_pos'] = total_pos
    sub_df['Weight_pos'] = weight_pos
    out_df = out_df.append(sub_df)

out_df.to_excel('enhanced.xlsx')

100%|████████████████████████████████████████████████████████████████████████████████| 161/161 [00:03<00:00, 43.46it/s]


In [6]:
out_df[['Total_pos', 'Weight_pos']]

,Total_pos,Weight_pos
Date,,
2007-01-03,"[0, 0, 0, 23.75, 25.0, 25.0, 1.25]","[0.0, 0.0, 0.0, 0.31666666666666665, 0.3333333..."
2007-01-04,"[0, 0, 0, 22.5, 25.0, 25.0, 2.5]","[0.0, 0.0, 0.0, 0.3, 0.3333333333333333, 0.333..."
2007-01-05,"[0, 0, 0, 21.25, 25.0, 25.0, 3.75]","[0.0, 0.0, 0.0, 0.2833333333333333, 0.33333333..."
2007-01-08,"[10.0, 12.5, 2.5, 20.0, 25.0, 25.0, 5.0]","[-0.1, -0.125, -0.025, 0.2, 0.25, 0.25, 0.05]"
2007-01-09,"[18.75, 25.0, 6.25, 20.625, 27.5, 27.5, 6.875]","[-0.14150943396226415, -0.18867924528301888, -..."
...,...,...
2019-12-24,"[7.142857142857146, 37.5, 30.357142857142854, ...","[-0.04761904761904764, -0.25, -0.2023809523809..."
2019-12-26,"[5.357142857142861, 37.5, 32.14285714285714, 3...","[-0.03571428571428574, -0.25, -0.2142857142857..."
2019-12-27,"[3.5714285714285765, 37.5, 33.92857142857142, ...","[-0.023809523809523843, -0.25, -0.226190476190..."
